### Atis nmt evaluation

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys

sys.path.append("../")

warnings.filterwarnings("ignore")

Download atis dataset from [here](https://github.com/Microsoft/CNTK/tree/master/Examples/LanguageUnderstanding/ATIS/Data)

### Run NER model

In [2]:
import os


data_path = "/datadrive/JointSLU/data/"
train_path = os.path.join(data_path, "train_filtered.csv")
valid_path = os.path.join(data_path, "valid_filtered.csv")
model_dir = " /datadrive/models/multi_cased_L-12_H-768_A-12/"
init_checkpoint_pt = os.path.join("/datadrive/models/multi_cased_L-12_H-768_A-12/", "pytorch_model.bin")
bert_config_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "bert_config.json")
vocab_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "vocab.txt")

In [3]:
import torch
torch.cuda.set_device(0)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 0)

#### Create data loaders

In [4]:
from modules import BertNerData as NerData

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [5]:
data = NerData.create(train_path, valid_path, vocab_file, data_type="bert_cased")

In [6]:
len(data.train_dl.dataset), len(data.valid_dl.dataset)

(9445, 888)

In [7]:
len(data.label2idx) #, len(data.cls2idx)

154

#### Create Ner model

Set params of encoder and decoder as proposed [here](https://arxiv.org/pdf/1609.01454.pdf)

In [22]:
from modules.models.bert_models import BertBiLSTMAttnNMT

In [23]:
model = BertBiLSTMAttnNMT.create(len(data.label2idx), bert_config_file, init_checkpoint_pt,
                                 enc_hidden_dim=256, dec_hidden_dim=256)

In [24]:
model.get_n_trainable_params()

1928231

In [25]:
sup_labels = list(pd.read_csv("/datadrive/JointSLU/data/slt_flt.csv").slots)

In [26]:
len(sup_labels)

106

#### Create learner

In [27]:
from modules import NerLearner

In [28]:
num_epochs = 250
learner = NerLearner(model, data,
                     best_model_path="/datadrive/models/atis/attn_final_nmt.cpt",
                     lr=0.01, clip=1.0, sup_labels=sup_labels,
                     t_total=num_epochs * len(data.train_dl))

INFO:root:Don't use lr scheduler...


In [ ]:
learner.fit(num_epochs, target_metric='prec')

### Get best results

In [30]:
learner.load_model()

#### Get span results for valid ds (where train support > 3)

In [16]:
import pandas as pd
sup_slots = list(pd.read_csv("/datadrive/JointSLU/data/sup_slots.csv").sup_slots)

In [31]:
from modules.data.bert_data import get_bert_data_loader_for_predict

In [32]:
dl = get_bert_data_loader_for_predict(data_path + "valid_filtered.csv", learner)

In [33]:
preds = learner.predict(dl)

In [35]:
from modules.train.train import validate_step


rep = validate_step(learner.data.valid_dl, learner.model, learner.data.id2label, learner.sup_labels)

Mean IOB precision

In [37]:
import numpy as np


np.mean([float(line.split()[1]) for line in rep.split("\n")[2:-5] if int(line.split()[-1]) > 0])

0.9191981132075471

In [39]:
from modules.utils.plot_metrics import get_bert_span_report


clf_report = get_bert_span_report(dl, preds, learner.sup_labels)
print(clf_report)

                            precision    recall  f1-score   support

                       mod      1.000     0.500     0.667         2
 return-date.date-relative      1.000     0.333     0.500         3
      arrive-date.day-name      0.714     0.909     0.800        11
                   connect      1.000     1.000     1.000         6
               compartment      0.000     0.000     0.000         1
                   economy      1.000     0.333     0.500         6
           toloc.city-name      0.955     0.980     0.967       712
    depart-time.period-mod      0.833     1.000     0.909         5
                      meal      0.941     1.000     0.970        16
        fromloc.state-name      1.000     1.000     1.000        17
                 days-code      1.000     1.000     1.000         1
              airline-name      0.959     0.979     0.969        96
                  day-name      1.000     0.500     0.667         2
           fare-basis-code      0.941     0.941